# Igbo Language Model Training - FIXED VERSION

**CRITICAL FIX:** Added proper checkpoint configuration

**Changes:**
- Added `checkpoint_s3_uri` to estimator
- Updated training script with checkpoint validation
- Changed save_total_limit to None (keep all checkpoints)
- Added backup model save to output_data_dir

**Configuration:**
- Model: Mistral-7B-v0.1
- Instance: ml.g5.xlarge (on-demand)
- Duration: ~7 days
- Cost: ~$237

In [1]:
import sagemaker
from sagemaker.huggingface import HuggingFace
from datetime import datetime
import boto3

print("✓ Imports loaded")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
✓ Imports loaded


In [2]:
# Configuration
BUCKET_NAME = 'learn-igbo-ekpes-useast1'
MODEL_NAME = 'mistralai/Mistral-7B-v0.1'

train_data_s3 = f's3://{BUCKET_NAME}/datasets/nllb/nllb_train.jsonl'
val_data_s3 = f's3://{BUCKET_NAME}/datasets/nllb/nllb_val.jsonl'
output_path = f's3://{BUCKET_NAME}/models/igbo-llm'
checkpoint_s3_uri = f's3://{BUCKET_NAME}/checkpoints/igbo-llm'  # NEW: Checkpoint location

timestamp = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
job_name = f'igbo-nllb-fixed-{timestamp}'

print("=" * 60)
print("CONFIGURATION - FIXED VERSION")
print("=" * 60)
print(f"Region: us-east-1")
print(f"Model: {MODEL_NAME}")
print(f"Training: {train_data_s3}")
print(f"Validation: {val_data_s3}")
print(f"Output: {output_path}")
print(f"Checkpoints: {checkpoint_s3_uri}")  # NEW
print(f"Job name: {job_name}")

CONFIGURATION - FIXED VERSION
Region: us-east-1
Model: mistralai/Mistral-7B-v0.1
Training: s3://learn-igbo-ekpes-useast1/datasets/nllb/nllb_train.jsonl
Validation: s3://learn-igbo-ekpes-useast1/datasets/nllb/nllb_val.jsonl
Output: s3://learn-igbo-ekpes-useast1/models/igbo-llm
Checkpoints: s3://learn-igbo-ekpes-useast1/checkpoints/igbo-llm
Job name: igbo-nllb-fixed-2026-01-24-02-16-53


In [3]:
# Hyperparameters - Same as before
hyperparameters = {
    'model_name': 'mistralai/Mistral-7B-v0.1',
    'epochs': 3,
    'learning_rate': 2e-4,
    'per_device_train_batch_size': 1,
    'gradient_accumulation_steps': 16,
    'max_length': 256,
    'lora_r': 16,
    'lora_alpha': 32,
    'lora_dropout': 0.05,
}

print("Hyperparameters:")
for k, v in hyperparameters.items():
    print(f"  {k}: {v}")

Hyperparameters:
  model_name: mistralai/Mistral-7B-v0.1
  epochs: 3
  learning_rate: 0.0002
  per_device_train_batch_size: 1
  gradient_accumulation_steps: 16
  max_length: 256
  lora_r: 16
  lora_alpha: 32
  lora_dropout: 0.05


In [4]:
# Initialize SageMaker session
session = sagemaker.Session()
role = sagemaker.get_execution_role()

print("\n" + "=" * 60)
print("CREATING SAGEMAKER ESTIMATOR - FIXED VERSION")
print("=" * 60)

# FIXED: HuggingFace estimator with checkpoint_s3_uri
estimator = HuggingFace(
    entry_point='train_igbo_model_FIXED.py',  # NEW: Fixed training script
    source_dir='.',  # Update this if your scripts are in a different location
    instance_type='ml.g5.xlarge',
    instance_count=1,
    role=role,
    sagemaker_session=session,
    transformers_version='4.46',
    pytorch_version='2.3',
    py_version='py311',
    hyperparameters=hyperparameters,
    output_path=output_path,
    checkpoint_s3_uri=checkpoint_s3_uri,  # NEW: Critical for checkpoint saving!
    checkpoint_local_path='/opt/ml/checkpoints',  # NEW: Local checkpoint directory
    base_job_name='igbo-nllb-fixed',
    max_run=604800,  # 7 days
    use_spot_instances=False,  # On-demand for reliability
    keep_alive_period_in_seconds=0,
)

print("✅ Estimator created!")
print(f"\nKey settings:")
print(f"  Entry point: train_igbo_model_FIXED.py")
print(f"  Instance: ml.g5.xlarge (on-demand)")
print(f"  Max runtime: 7 days")
print(f"  Output: {output_path}")
print(f"  Checkpoints: {checkpoint_s3_uri}")  # NEW
print(f"  Checkpoint sync: ENABLED ✓")  # NEW


CREATING SAGEMAKER ESTIMATOR - FIXED VERSION
✅ Estimator created!

Key settings:
  Entry point: train_igbo_model_FIXED.py
  Instance: ml.g5.xlarge (on-demand)
  Max runtime: 7 days
  Output: s3://learn-igbo-ekpes-useast1/models/igbo-llm
  Checkpoints: s3://learn-igbo-ekpes-useast1/checkpoints/igbo-llm
  Checkpoint sync: ENABLED ✓


In [10]:
# Launch training
confirm = input("Type 'START' to launch FIXED training (~$237, 7 days): ")

if confirm == 'START':
    print("\n🚀 Launching FIXED training job...")
    
    estimator.fit(
        inputs={'train': train_data_s3, 'validation': val_data_s3},
        wait=False
    )
    
    job_name = estimator.latest_training_job.name
    region = session.boto_region_name
    
    print("\n" + "=" * 60)
    print("✅ TRAINING JOB LAUNCHED - FIXED VERSION!")
    print("=" * 60)
    print(f"\nJob name: {job_name}")
    print(f"Region: {region}")
    print(f"Instance: ml.g5.xlarge (on-demand)")
    
    print(f"\n📊 Monitor at:")
    print(f"https://console.aws.amazon.com/sagemaker/home?region={region}#/jobs/{job_name}")
    
    print(f"\n📝 CloudWatch Logs:")
    print(f"https://console.aws.amazon.com/cloudwatch/home?region={region}#logStream:group=/aws/sagemaker/TrainingJobs;prefix={job_name}")
    
    print(f"\n📦 Checkpoints will be saved to:")
    print(f"{checkpoint_s3_uri}/{job_name}/checkpoints/")
    
    print(f"\n✅ Training will start within 5-10 minutes")
    print(f"✅ Expected completion: ~7 days from now")
    print(f"✅ Checkpoints: ENABLED - saves every 5000 steps")
    print(f"✅ No checkpoint deletion - all saved!")
    
    print(f"\n💰 Total cost: ~$237")
    
    print("\n🎉 You can close this notebook - training runs independently!")
    print("=" * 60)
    
    # Validation reminder
    print("\n⚠️  IMPORTANT: Validate checkpoint saving!")
    print("   After 30 minutes, check S3 for first checkpoint:")
    print(f"   aws s3 ls {checkpoint_s3_uri}/{job_name}/checkpoints/ --recursive")
    print("   You should see checkpoint-5000/ folder after ~90 minutes")
    
else:
    print("❌ Cancelled - nothing was launched")

Type 'START' to launch FIXED training (~$237, 7 days):  START


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.



🚀 Launching FIXED training job...


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: igbo-nllb-fixed-2026-01-24-04-23-30-627



✅ TRAINING JOB LAUNCHED - FIXED VERSION!

Job name: igbo-nllb-fixed-2026-01-24-04-23-30-627
Region: us-east-1
Instance: ml.g5.xlarge (on-demand)

📊 Monitor at:
https://console.aws.amazon.com/sagemaker/home?region=us-east-1#/jobs/igbo-nllb-fixed-2026-01-24-04-23-30-627

📝 CloudWatch Logs:
https://console.aws.amazon.com/cloudwatch/home?region=us-east-1#logStream:group=/aws/sagemaker/TrainingJobs;prefix=igbo-nllb-fixed-2026-01-24-04-23-30-627

📦 Checkpoints will be saved to:
s3://learn-igbo-ekpes-useast1/checkpoints/igbo-llm/igbo-nllb-fixed-2026-01-24-04-23-30-627/checkpoints/

✅ Training will start within 5-10 minutes
✅ Expected completion: ~7 days from now
✅ Checkpoints: ENABLED - saves every 5000 steps
✅ No checkpoint deletion - all saved!

💰 Total cost: ~$237

🎉 You can close this notebook - training runs independently!

⚠️  IMPORTANT: Validate checkpoint saving!
   After 30 minutes, check S3 for first checkpoint:
   aws s3 ls s3://learn-igbo-ekpes-useast1/checkpoints/igbo-llm/igbo-n

In [9]:
# VALIDATION: Check if checkpoint saving is working
# Run this after ~90 minutes (after first checkpoint at step 5000)
import boto3

s3_client = boto3.client('s3', region_name='us-east-1')

try:
    job_name = estimator.latest_training_job.name
    checkpoint_prefix = f"checkpoints/igbo-llm/{job_name}/checkpoints/"
    
    print("Checking for checkpoints...")
    print(f"Location: s3://{BUCKET_NAME}/{checkpoint_prefix}")
    print("\n" + "=" * 60)
    
    response = s3_client.list_objects_v2(
        Bucket=BUCKET_NAME,
        Prefix=checkpoint_prefix
    )
    
    if 'Contents' in response:
        checkpoints = {}
        for obj in response['Contents']:
            if 'checkpoint-' in obj['Key']:
                checkpoint_num = obj['Key'].split('checkpoint-')[1].split('/')[0]
                if checkpoint_num not in checkpoints:
                    checkpoints[checkpoint_num] = []
                checkpoints[checkpoint_num].append(obj['Key'].split('/')[-1])
        
        if checkpoints:
            print(f"✅ Found {len(checkpoints)} checkpoint(s)!")
            for cp_num in sorted(checkpoints.keys(), key=int):
                print(f"\nCheckpoint {cp_num}:")
                for file in sorted(checkpoints[cp_num]):
                    print(f"  ✓ {file}")
        else:
            print("⚠️  No checkpoints found yet")
            print("   Wait longer and try again")
    else:
        print("⚠️  No checkpoints found yet")
        print("   Wait for first checkpoint at step 5000 (~90 minutes)")
    
except Exception as e:
    print(f"Error: {e}")
    print("\nMake sure you've launched training first (run Cell 6)")

Checking for checkpoints...
Location: s3://learn-igbo-ekpes-useast1/checkpoints/igbo-llm/igbo-nllb-fixed-2026-01-24-02-17-36-460/checkpoints/

⚠️  No checkpoints found yet
   Wait for first checkpoint at step 5000 (~90 minutes)


In [7]:
# Optional: Check job status
import boto3

sm_client = boto3.client('sagemaker', region_name='us-east-1')

try:
    job_name = estimator.latest_training_job.name
    response = sm_client.describe_training_job(TrainingJobName=job_name)
    
    print(f"Job: {job_name}")
    print(f"Status: {response['TrainingJobStatus']}")
    print(f"Secondary Status: {response['SecondaryStatus']}")
    
    if 'BillableTimeInSeconds' in response:
        billable_hours = response['BillableTimeInSeconds'] / 3600
        cost = billable_hours * 1.41
        print(f"Billable time: {billable_hours:.1f} hours")
        print(f"Cost so far: ${cost:.2f}")
        
        # Estimate completion
        if billable_hours > 0:
            hours_per_epoch = 58.7  # From previous run
            epochs_completed = billable_hours / hours_per_epoch
            print(f"Estimated epochs completed: {epochs_completed:.2f} / 3.0")
            
except Exception as e:
    print(f"Error: {e}")
    print("No training job found. Run Cell 6 first to launch training.")

Job: igbo-nllb-fixed-2026-01-24-02-17-36-460
Status: InProgress
Secondary Status: Training
